In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load training data
X_train = pd.read_csv('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/train/features/cluster_features.csv')
y_train = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/train/labels/labels.npy')

# Load validation data
X_val = pd.read_csv('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/val/features/cluster_features.csv')
y_val = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/val/labels/labels.npy')

# Load test data (no labels)
X_test = pd.read_csv('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/features/cluster_features.csv')
test_ids = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy')


# DEEP NEURAL NETWORK (DNN) - STANDARD

In [3]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [4]:
'''
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Build DNN
dnn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

# Compile
dnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['AUC']
)

# Callbacks
early_stop = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max')

# Train
history = dnn.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)
'''

"\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout\nfrom tensorflow.keras.callbacks import EarlyStopping\n\n# Build DNN\ndnn = Sequential([\n    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),\n    Dropout(0.3),\n    Dense(64, activation='relu'),\n    Dropout(0.3),\n    Dense(32, activation='relu'),\n    Dense(1, activation='sigmoid')  # Binary output\n])\n\n# Compile\ndnn.compile(\n    optimizer='adam',\n    loss='binary_crossentropy',\n    metrics=['AUC']\n)\n\n# Callbacks\nearly_stop = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max')\n\n# Train\nhistory = dnn.fit(\n    X_train_scaled, y_train,\n    validation_data=(X_val_scaled, y_val),\n    epochs=100,\n    batch_size=128,\n    callbacks=[early_stop],\n    verbose=1\n)\n"

In [5]:
'''
# Plot training and validation AUC
plt.figure(figsize=(8, 5))
plt.plot(history.history['AUC'], label='Train AUC')
plt.plot(history.history['val_AUC'], label='Validation AUC')
plt.title('Training vs Validation AUC Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.grid(True)
plt.show()
'''

"\n# Plot training and validation AUC\nplt.figure(figsize=(8, 5))\nplt.plot(history.history['AUC'], label='Train AUC')\nplt.plot(history.history['val_AUC'], label='Validation AUC')\nplt.title('Training vs Validation AUC Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('AUC')\nplt.legend()\nplt.grid(True)\nplt.show()\n"

In [6]:
'''
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Predict probabilities on validation set
val_preds_dnn = dnn.predict(X_val_scaled).flatten()

# Compute ROC AUC
auc_dnn = roc_auc_score(y_val, val_preds_dnn)
print(f"✅ Best Model: DNN | Validation AUC: {auc_dnn:.4f}")

# Compute ROC curve
fpr, tpr, _ = roc_curve(y_val, val_preds_dnn)

# Plot ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'DNN (AUC = {auc_dnn:.4f})', color='darkorange')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Best Model (DNN)")
plt.legend()
plt.grid(True)
plt.show()
'''

'\nfrom sklearn.metrics import roc_auc_score, roc_curve\nimport matplotlib.pyplot as plt\n\n# Predict probabilities on validation set\nval_preds_dnn = dnn.predict(X_val_scaled).flatten()\n\n# Compute ROC AUC\nauc_dnn = roc_auc_score(y_val, val_preds_dnn)\nprint(f"✅ Best Model: DNN | Validation AUC: {auc_dnn:.4f}")\n\n# Compute ROC curve\nfpr, tpr, _ = roc_curve(y_val, val_preds_dnn)\n\n# Plot ROC\nplt.figure(figsize=(8, 6))\nplt.plot(fpr, tpr, label=f\'DNN (AUC = {auc_dnn:.4f})\', color=\'darkorange\')\nplt.plot([0, 1], [0, 1], linestyle=\'--\', color=\'gray\')\nplt.xlabel("False Positive Rate")\nplt.ylabel("True Positive Rate")\nplt.title("ROC Curve – Best Model (DNN)")\nplt.legend()\nplt.grid(True)\nplt.show()\n'

In [7]:
'''
np.save('/kaggle/working/03_dnn_preds.npy', val_preds_dnn)

# Scale test data using the same StandardScaler
X_test_scaled = scaler.transform(X_test)

# Predict on scaled test data
test_dnn_preds = dnn.predict(X_test_scaled, batch_size=512).flatten()
np.save('/kaggle/working/03_dnn_test_preds.npy', test_dnn_preds)
'''

"\nnp.save('/kaggle/working/03_dnn_preds.npy', val_preds_dnn)\n\n# Scale test data using the same StandardScaler\nX_test_scaled = scaler.transform(X_test)\n\n# Predict on scaled test data\ntest_dnn_preds = dnn.predict(X_test_scaled, batch_size=512).flatten()\nnp.save('/kaggle/working/03_dnn_test_preds.npy', test_dnn_preds)\n"

# Standard DNN - Submission

In [8]:
'''
# Load test IDs and DNN test predictions
test_ids = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy')
test_preds_dnn = np.load('/kaggle/working/03_dnn_test_preds.npy')

# Create submission DataFrame
submission_dnn = pd.DataFrame({
    'id': test_ids,
    'label': test_preds_dnn
})

# Save to CSV
submission_dnn.to_csv('/kaggle/working/submission.csv', index=False)
print("DNN submission file saved: submission.csv")
'''

'\n# Load test IDs and DNN test predictions\ntest_ids = np.load(\'/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy\')\ntest_preds_dnn = np.load(\'/kaggle/working/03_dnn_test_preds.npy\')\n\n# Create submission DataFrame\nsubmission_dnn = pd.DataFrame({\n    \'id\': test_ids,\n    \'label\': test_preds_dnn\n})\n\n# Save to CSV\nsubmission_dnn.to_csv(\'/kaggle/working/submission.csv\', index=False)\nprint("DNN submission file saved: submission.csv")\n'

# Feature Engineering

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Copying for Feature Engineering

X_train_fe = X_train.copy()
X_val_fe = X_val.copy()
X_test_fe = X_test.copy()

# 1. Log transform of total_pt — reduces skew and handles large scale
X_train_fe['log_total_pt'] = np.log1p(X_train_fe['total_pt'])
X_val_fe['log_total_pt'] = np.log1p(X_val_fe['total_pt'])
X_test_fe['log_total_pt'] = np.log1p(X_test_fe['total_pt'])

# 2. pt per cluster — normalizes energy by cluster count
X_train_fe['pt_per_cluster'] = X_train_fe['total_pt'] / (X_train_fe['n_clusters'] + 1e-5)
X_val_fe['pt_per_cluster'] = X_val_fe['total_pt'] / (X_val_fe['n_clusters'] + 1e-5)
X_test_fe['pt_per_cluster'] = X_test_fe['total_pt'] / (X_test_fe['n_clusters'] + 1e-5)

# 3. pt to cluster size ratio — captures energy concentration
X_train_fe['pt_size_ratio'] = X_train_fe['mean_cluster_pt'] / (X_train_fe['mean_cluster_size'] + 1e-5)
X_val_fe['pt_size_ratio'] = X_val_fe['mean_cluster_pt'] / (X_val_fe['mean_cluster_size'] + 1e-5)
X_test_fe['pt_size_ratio'] = X_test_fe['mean_cluster_pt'] / (X_test_fe['mean_cluster_size'] + 1e-5)

# 4. Eta-phi spread — measures spatial dispersion
X_train_fe['eta_phi_spread'] = (X_train_fe['max_cluster_eta'] - X_train_fe['mean_cluster_eta']) * \
                               (X_train_fe['max_cluster_phi'] - X_train_fe['mean_cluster_phi'])
X_val_fe['eta_phi_spread'] = (X_val_fe['max_cluster_eta'] - X_val_fe['mean_cluster_eta']) * \
                             (X_val_fe['max_cluster_phi'] - X_val_fe['mean_cluster_phi'])
X_test_fe['eta_phi_spread'] = (X_test_fe['max_cluster_eta'] - X_test_fe['mean_cluster_eta']) * \
                              (X_test_fe['max_cluster_phi'] - X_test_fe['mean_cluster_phi'])

# --------------------------------------------
# ✳️ Standard Scaling for DNN
# --------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_fe)
X_val_scaled = scaler.transform(X_val_fe)
X_test_scaled = scaler.transform(X_test_fe)

print("Done: Feature engineering and scaling applied")


Done: Feature engineering and scaling applied


# DNN - After Feature Engineering

In [10]:
'''
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Build a deeper and more regularized DNN
dnn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile with AUC as the key metric
dnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='AUC')]
)

# Early stopping to avoid overfitting
early_stop = EarlyStopping(
    monitor='val_AUC', patience=10, restore_best_weights=True, mode='max'
)

# Train the model
history = dnn.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)
'''

"\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout, BatchNormalization\nfrom tensorflow.keras.callbacks import EarlyStopping\n\n# Build a deeper and more regularized DNN\ndnn = Sequential([\n    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),\n    BatchNormalization(),\n    Dropout(0.3),\n\n    Dense(64, activation='relu'),\n    BatchNormalization(),\n    Dropout(0.3),\n\n    Dense(32, activation='relu'),\n    Dense(1, activation='sigmoid')  # Binary classification\n])\n\n# Compile with AUC as the key metric\ndnn.compile(\n    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),\n    loss='binary_crossentropy',\n    metrics=[tf.keras.metrics.AUC(name='AUC')]\n)\n\n# Early stopping to avoid overfitting\nearly_stop = EarlyStopping(\n    monitor='val_AUC', patience=10, restore_best_weights=True, mode='max'\n)\n\n# Train the model\nhistory = dnn.fit(\n    X_train_scaled, y_train,\n    val

In [11]:
'''
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(history.history['AUC'], label='Train AUC')
plt.plot(history.history['val_AUC'], label='Validation AUC')
plt.title('Training vs Validation AUC Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.grid(True)
plt.show()
'''

"\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(8, 5))\nplt.plot(history.history['AUC'], label='Train AUC')\nplt.plot(history.history['val_AUC'], label='Validation AUC')\nplt.title('Training vs Validation AUC Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('AUC')\nplt.legend()\nplt.grid(True)\nplt.show()\n"

In [12]:
'''
from sklearn.metrics import roc_auc_score, roc_curve

# Validation predictions
val_preds_dnn = dnn.predict(X_val_scaled).flatten()
val_auc_dnn = roc_auc_score(y_val, val_preds_dnn)
print(f"✅ Final Validation AUC (DNN): {val_auc_dnn:.5f}")

# Test predictions
test_preds_dnn = dnn.predict(X_test_scaled, batch_size=512).flatten()

# Save .npy if needed
np.save('/kaggle/working/002dnn_val_preds.npy', val_preds_dnn)
np.save('/kaggle/working/002dnn_test_preds.npy', test_preds_dnn)
'''

'\nfrom sklearn.metrics import roc_auc_score, roc_curve\n\n# Validation predictions\nval_preds_dnn = dnn.predict(X_val_scaled).flatten()\nval_auc_dnn = roc_auc_score(y_val, val_preds_dnn)\nprint(f"✅ Final Validation AUC (DNN): {val_auc_dnn:.5f}")\n\n# Test predictions\ntest_preds_dnn = dnn.predict(X_test_scaled, batch_size=512).flatten()\n\n# Save .npy if needed\nnp.save(\'/kaggle/working/002dnn_val_preds.npy\', val_preds_dnn)\nnp.save(\'/kaggle/working/002dnn_test_preds.npy\', test_preds_dnn)\n'

# DNN with Hyperparameter Tuning

In [13]:
'''
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define a function to build the model with parameters
def build_model(input_dim, units=128, dropout=0.3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(dropout),
        Dense(units//2, activation='relu'),
        BatchNormalization(),
        Dropout(dropout),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='AUC')]
    )
    return model

# Define hyperparameter grid
param_grid = [
    {'units': 128, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 128},
    {'units': 128, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 128},
    {'units': 128, 'dropout': 0.3, 'learning_rate': 0.0005, 'batch_size': 128},
    {'units': 64,  'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 128},
    {'units': 64,  'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 128},
    {'units': 128, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64},
    {'units': 128, 'dropout': 0.4, 'learning_rate': 0.0005, 'batch_size': 64},
    {'units': 64,  'dropout': 0.3, 'learning_rate': 0.0005, 'batch_size': 64},
]

# Setup KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

# Start grid search with cross-validation
for params in param_grid:
    print(f"\n🔍 Testing config: {params}")
    fold_aucs = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled)):
        X_tr, X_val_cv = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val_cv = y_train[train_idx], y_train[val_idx]

        model = build_model(
            input_dim=X_tr.shape[1],
            units=params['units'],
            dropout=params['dropout'],
            learning_rate=params['learning_rate']
        )

        early_stop = EarlyStopping(
            monitor='val_AUC', patience=10, restore_best_weights=True, mode='max'
        )

        model.fit(
            X_tr, y_tr,
            validation_data=(X_val_cv, y_val_cv),
            epochs=100,
            batch_size=params['batch_size'],
            callbacks=[early_stop],
            verbose=0
        )

        preds = model.predict(X_val_cv, batch_size=512).flatten()
        auc = roc_auc_score(y_val_cv, preds)
        fold_aucs.append(auc)
        print(f"  Fold {fold+1} AUC: {auc:.5f}")

    avg_auc = np.mean(fold_aucs)
    std_auc = np.std(fold_aucs)
    results.append({
        'params': params,
        'avg_auc': avg_auc,
        'std_auc': std_auc
    })
    print(f"✅ Avg AUC: {avg_auc:.5f} | Std: {std_auc:.5f}")

# Sort results by highest average AUC
sorted_results = sorted(results, key=lambda x: x['avg_auc'], reverse=True)
best_result = sorted_results[0]

import pandas as pd
df_cv_results = pd.DataFrame(sorted_results)
'''

'\nimport tensorflow as tf\nfrom sklearn.model_selection import KFold\nfrom sklearn.metrics import roc_auc_score\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout, BatchNormalization\nfrom tensorflow.keras.callbacks import EarlyStopping\n\n# Define a function to build the model with parameters\ndef build_model(input_dim, units=128, dropout=0.3, learning_rate=0.001):\n    model = Sequential([\n        Dense(units, activation=\'relu\', input_shape=(input_dim,)),\n        BatchNormalization(),\n        Dropout(dropout),\n        Dense(units//2, activation=\'relu\'),\n        BatchNormalization(),\n        Dropout(dropout),\n        Dense(1, activation=\'sigmoid\')\n    ])\n    model.compile(\n        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),\n        loss=\'binary_crossentropy\',\n        metrics=[tf.keras.metrics.AUC(name=\'AUC\')]\n    )\n    return model\n\n# Define hyperparameter grid\nparam_grid = [\n    {\'unit

In [14]:
# Show the results directly
#df_cv_results

# DNN - Final Training with best config

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Final model
final_dnn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

final_dnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['AUC']
)

early_stop = EarlyStopping(monitor='val_AUC', patience=10, restore_best_weights=True, mode='max')

# Train on full training data, validate on your validation split
final_dnn.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)

2025-06-20 07:05:08.289722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750403108.592498      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750403108.677005      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-20 07:05:24.780038: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - AUC: 0.7225 - loss: 0.7190 - val_AUC: 0.9280 - val_loss: 0.4710
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9108 - loss: 0.4216 - val_AUC: 0.9356 - val_loss: 0.4395
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9222 - loss: 0.3695 - val_AUC: 0.9360 - val_loss: 0.4177
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9295 - loss: 0.3421 - val_AUC: 0.9407 - val_loss: 0.3938
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9257 - loss: 0.3404 - val_AUC: 0.9418 - val_loss: 0.3759
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9245 - loss: 0.3356 - val_AUC: 0.9417 - val_loss: 0.3581
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9257 - loss: 0.3334 - val_AUC: 0.9442 - val_loss: 0.3417
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9344 - loss: 0.3061 - val_AUC: 0.9458 - val_loss: 0.3254
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.9298 - loss: 0.

In [16]:
# Evaluate
val_preds = final_dnn.predict(X_val_scaled, batch_size=512).flatten()
from sklearn.metrics import roc_auc_score
val_auc = roc_auc_score(y_val, val_preds)
print(f"✅ Final Model Validation AUC: {val_auc:.5f}")

# Predict on test
test_preds = final_dnn.predict(X_test_scaled, batch_size=512).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
✅ Final Model Validation AUC: 0.94637
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 


In [17]:
# Save submission
submission = pd.DataFrame({
    'id': test_ids,
    'label': test_preds
})
submission.to_csv('/kaggle/working/submission.csv', index=False)
print("🚀 Submission saved as: submission.csv")

🚀 Submission saved as: submission.csv


In [18]:
# Save as .npy for ensembling
np.save('/kaggle/working/02_dnn_tuned_test_preds.npy', test_preds)

print("Saved DNN test predictions as 02_dnn_tuned_test_preds.npy")

Saved DNN test predictions as 02_dnn_tuned_test_preds.npy


In [19]:
from sklearn.metrics import roc_auc_score
import numpy as np

# Predict on validation set using final DNN
val_preds_dnn = final_dnn.predict(X_val_scaled, batch_size=512).flatten()

# Calculate and display AUC
val_auc_dnn = roc_auc_score(y_val, val_preds_dnn)
print(f"✅ Final DNN Validation AUC: {val_auc_dnn:.5f}")

# Save predictions as .npy file for ensembling
np.save('/kaggle/working/dnn_tuned_val_preds.npy', val_preds_dnn)

print("💾 Saved DNN validation predictions: dnn_tuned_val_preds.npy")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
✅ Final DNN Validation AUC: 0.94637
💾 Saved DNN validation predictions: dnn_tuned_val_preds.npy
